# 강우관측소와 동일한 위치의 천리안 위성자료 취득

## 1. 관련 Python 라이브러리 설정

In [1]:
import sys
import platform
import random
import math
import gc
import joblib
from glob import glob
import datetime
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import netCDF4
import xarray as xr
 
import dateutil
from glob import glob
from datetime import timedelta

# import warnings
# warnings.filterwarnings(action='ignore')
 
print(f"- os: {platform.platform()}")
print(f"- python: {sys.version}")
print(f"- pandas: {pd.__version__}")
print(f"- numpy: {np.__version__}")

- os: Linux-5.4.0-91-generic-x86_64-with-debian-buster-sid
- python: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
- pandas: 1.3.5
- numpy: 1.21.5


## 2. 설정된 폴더의 RR과 TPW자료의 리스트 취득

In [3]:
rr_path_list = sorted( glob(f'./data/RR/*.nc') , key = lambda x : x.split('/')[-1] )
tpw_path_list = sorted( glob(f'./data/TPW/*.nc') , key = lambda x : x.split('/')[-1] )

rr_nc_keys = ['RR']
tpw_nc_keys = ['LPW1', 'LPW2', 'LPW3', 'TPW']

len(rr_path_list) , len(tpw_path_list)

(137, 142)

## 3. 관측 전기간에 대한 10분단위 시간자료 생성

* 본 자료는 10분단위 전체기간을 나타내며 아래에서 천리안 위성자료의 결측자료와 비교시 활용

In [4]:
date_df = pd.DataFrame(
    {'date': pd.date_range(start='2020-04-01 00:00:00', end='2020-04-01 23:50:00', freq='10min'),
    }
)
date_df

,date
0,2020-04-01 00:00:00
1,2020-04-01 00:10:00
2,2020-04-01 00:20:00
3,2020-04-01 00:30:00
4,2020-04-01 00:40:00
...,...
139,2020-04-01 23:10:00
140,2020-04-01 23:20:00
141,2020-04-01 23:30:00
142,2020-04-01 23:40:00


## 4. 위에서 생성한 시간(date)자료에 천리안자료 추가를 위한 함수생성

In [5]:
def add_path_column(df , path_list , col_name = ""):
    path_series = []
    dt_series = []
    df = df.copy()
    for path in path_list:
        datetimeobj = datetime.datetime(int(path.split('\\')[-1][-15:-11]), int(path.split('\\')[-1][-11:-9]), int(path.split('\\')[-1][-9:-7]), int(path.split('\\')[-1][-7:-5]), int(path.split('\\')[-1][-5:-3]))
        dt_series.append(datetimeobj)
        path_series.append(path)

    dict_ = {
        "date" : dt_series,
        col_name : path_series,
    }
    tmp = pd.DataFrame(dict_)
    df = pd.merge(df, tmp  ,how="left")
    df[col_name] = df[col_name].interpolate(method="pad")
    return df

In [6]:
x_train_df = add_path_column(date_df , rr_path_list , col_name = "rr_path")
x_train_df = add_path_column(x_train_df , tpw_path_list , col_name = "tpw_path")

In [7]:
x_train_df

,date,rr_path,tpw_path
0,2020-04-01 00:00:00,./data/RR/RR_202004010000.nc,./data/TPW/TPW_202004010000.nc
1,2020-04-01 00:10:00,./data/RR/RR_202004010010.nc,./data/TPW/TPW_202004010010.nc
2,2020-04-01 00:20:00,./data/RR/RR_202004010020.nc,./data/TPW/TPW_202004010020.nc
3,2020-04-01 00:30:00,./data/RR/RR_202004010030.nc,./data/TPW/TPW_202004010030.nc
4,2020-04-01 00:40:00,./data/RR/RR_202004010030.nc,./data/TPW/TPW_202004010030.nc
...,...,...,...
139,2020-04-01 23:10:00,./data/RR/RR_202004012310.nc,./data/TPW/TPW_202004012310.nc
140,2020-04-01 23:20:00,./data/RR/RR_202004012320.nc,./data/TPW/TPW_202004012320.nc
141,2020-04-01 23:30:00,./data/RR/RR_202004012330.nc,./data/TPW/TPW_202004012330.nc
142,2020-04-01 23:40:00,./data/RR/RR_202004012340.nc,./data/TPW/TPW_202004012340.nc


## 5. 천리안 강우강도 NetCDF 파일 확인

In [8]:
x_train_df['rr_path'][0]

'./data/RR/RR_202004010000.nc'

In [9]:
rr_sample = xr.open_dataset(x_train_df['rr_path'][0])
rr_sample

<xarray.Dataset>
Dimensions:                 (dim_y: 900, dim_x: 900)
Dimensions without coordinates: dim_y, dim_x
Data variables:
    RR                      (dim_y, dim_x) float32 ...
    Raining_CT_flag         (dim_y, dim_x) float32 ...
    gk2a_imager_projection  int32 ...
Attributes: (12/16)
    file_creation_time:      2020-04-01T00:06:15.77Z
    origianl_sourece_file:   gk2a_ami_le2_rr_ela020ge_202004010000.nc
    algorithm_version:       GK2A_RR_v1.6.0.4
    comment:                 TBD
    conventions:             CF-1.7
    institution:             KMA/NMSC> Korea Meteorological Administration, N...
    ...                      ...
    number_of_product:       2
    observation_mode:        ELA
    processing_area:         ELA
    processing_environment:  operation
    references:              RAP03 RR v1.0
    title:                   AMI L2 RR

## 6. 천리안 가강수량 NetCDF 파일 확인

In [10]:
x_train_df['tpw_path'][0]

'./data/TPW/TPW_202004010000.nc'

In [11]:
tpw_sample = xr.open_dataset(x_train_df['tpw_path'][0])
tpw_sample

<xarray.Dataset>
Dimensions:                 (dim_y: 900, dim_x: 900)
Dimensions without coordinates: dim_y, dim_x
Data variables:
    LPW1                    (dim_y, dim_x) float32 ...
    LPW2                    (dim_y, dim_x) float32 ...
    LPW3                    (dim_y, dim_x) float32 ...
    TPW                     (dim_y, dim_x) float32 ...
    quality_flag1           (dim_y, dim_x) float32 ...
    quality_flag2           (dim_y, dim_x) float32 ...
    quality_flag3           uint8 ...
    gk2a_imager_projection  int32 ...
Attributes: (12/16)
    file_creation_time:      2020-04-01T00:15:09.421Z
    origianl_sourece_file:   gk2a_ami_le2_tpw_fd060ge_202004010000.nc
    algorithm_version:       GK2A_AAP_v1.6.7.4
    comment:                 TBD
    conventions:             CF-1.7
    institution:             KMA/NMSC> Korea Meteorological Administration, N...
    ...                      ...
    number_of_product:       7
    observation_mode:        FD
    processing_area:         FD
    processing_environment:  operation
    references:              RAP03 AAP v1.3
    title:                   AMI L2 AAP

## 7. 천리안 관측 전지점 좌표정리

In [17]:
with open('gk2a_lcc_latlon_ko\\LATLON_KO_2000.txt') as f:
    lines = f.readlines()

lat_total = []
for value in lines:
    lat = value.split('\t')[0]
    lat_total.append(float(lat))
lat_total    

lon_total = []
for value in lines:
    lon = value.split('\t')[-1]
    lon_total.append(float(lon))
lon_total

## 8. 강우관측소와 동일한 위치의 천리안 자료 index 추출

In [12]:
from glob import glob
from PIL import Image

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import geopandas as gpd

In [13]:
# Load shape file

che_shp = gpd.read_file('../geometry/input-cheonlian-2A/천리안.shp')
out_shp = gpd.read_file('../geometry/output-seomjingang/섬진강강수량k.shp')

In [14]:
che_shp

,id,y,x,geometry
0,1,45.72897,113.99642,POINT (-811930.061 1541394.609)
1,2,45.73175,114.02273,POINT (-809833.092 1541365.296)
2,3,45.73452,114.04904,POINT (-807736.505 1541335.582)
3,4,45.73729,114.07535,POINT (-805640.118 1541306.578)
4,5,45.74005,114.10166,POINT (-803544.113 1541277.172)
...,...,...,...,...
809995,809996,29.32051,135.16518,POINT (994545.993 -334859.607)
809996,809997,29.31845,135.18557,POINT (996553.341 -334949.066)
809997,809998,29.31639,135.20596,POINT (998560.824 -335038.180)
809998,809999,29.31432,135.22635,POINT (1000568.519 -335128.063)


In [15]:
out_shp

,대권역,관측소,관측_1,관측_12,관측__13,국가표,댐명,x,y,geometry
0,섬진강,4001430.0,성수,진안군(도통리),Jinangun(Dotongri),40014070,섬진강,127.343125,35.712858,POINT (231049.237 246238.732)
1,섬진강,4001440.0,신평,임실군(용암리),Imsilgun(Yongamri),40014060,섬진강,127.188436,35.642942,POINT (217066.362 238443.349)
2,섬진강,4001450.0,쌍치,순창군(시산리),Sunchanggun(Sisanri),40014050,섬진강,126.971969,35.488703,POINT (197456.449 221314.402)
3,섬진강,4003420.0,섬진강댐,임실군(섬진강댐),Imsilgun(Seomjingangdam),40024020,섬진강,127.111697,35.542364,POINT (210128.899 227273.483)
4,섬진강,4007450.0,복래,보성군(복내리),Boseonggun(Boknaeri),40074060,주암(본),127.134581,34.896425,POINT (212301.077 155612.576)
5,섬진강,4007470.0,주암댐,순천시(주암댐),Suncheongsi(Juamdam),40074080,주암(본),127.235964,35.063556,POINT (221524.064 174171.272)
6,섬진강,4007472.0,동가,화순군(동가리),Hwasungun(Donggari),40074140,주암(본),127.065492,34.969314,POINT (205980.846 163692.516)
7,섬진강,4007474.0,맹리,화순군(맹리),Hwasungun(Maengri),40074143,주암(본),127.093417,35.178719,POINT (208509.254 186926.410)
8,섬진강,4009460.0,우산,순천시(우산리),Suncheongsi(Usanri),40074070,주암(본),127.222950,34.973844,POINT (220359.189 164215.882)
9,섬진강,9000140.0,동복,화순군(동복댐),Hwasungun(Dongbokdam),40074050,주암(본),127.100611,35.081975,POINT (209175.437 176193.928)


In [16]:
# 관측소 위치와 가장 가까운 천리안 위성자료 취득

neareast_idx = []
for i in range(len(out_shp)):
    neareast_idx.append(list(che_shp.distance(out_shp.loc[i,'geometry']).sort_values()[:1].index)[0])
neareast_idx

[560310,
 563903,
 571093,
 568400,
 600801,
 592705,
 597198,
 586399,
 596305,
 591799,
 606195,
 605302]

## 9. 천리안 위성자료를 관측소코드별 csv파일로 저장

In [17]:
# 원본 천리안 자료의 Dim 확인
rr_sample['RR'].shape

(900, 900)

In [18]:
# 8번에서 추출한 index에 맞는 데이터의 추출을 위하여 N:1의 배열로 수정
RR = np.reshape(rr_sample['RR'], (810000, 1))
RR

<xarray.DataArray 'RR' (dim_y: 810000, dim_x: 1)>
array([[0.       ],
       [0.       ],
       [0.       ],
       ...,
       [4.4      ],
       [4.99     ],
       [5.5899997]], dtype=float32)
Dimensions without coordinates: dim_y, dim_x

In [19]:
# 관측변수별 index를 활용한 array 포맷생성
sta_index = neareast_idx
sta_index_rr = RR[sta_index]*0
sta_index_lpw1 = RR[sta_index]*0
sta_index_lpw2 = RR[sta_index]*0
sta_index_lpw3 = RR[sta_index]*0
sta_index_tpw = RR[sta_index]*0

In [20]:
sta_index_rr

<xarray.DataArray 'RR' (dim_y: 12, dim_x: 1)>
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)
Dimensions without coordinates: dim_y, dim_x

In [21]:
# index을 활용한 pandas dataframe 생성
kma = pd.DataFrame({'index': sta_index})
kma

,index
0,560310
1,563903
2,571093
3,568400
4,600801
5,592705
6,597198
7,586399
8,596305
9,591799


* 관측소별 RR, TPW, LPW1, LPW2, LPW3 자료 저장
1년 12개 관측소 데이터 추출시 1시간 15분 소요

In [22]:
%%time
#rainfall_extract = []
for i in range(len(x_train_df)):
    x_train_df['rr_path'][i]
    rr_sample = xr.open_dataset(x_train_df['rr_path'][i])
    RR = np.reshape(rr_sample['RR'], (810000, 1))
    tpw_sample = xr.open_dataset(x_train_df['tpw_path'][i])
    LPW1 = np.reshape(tpw_sample['LPW1'], (810000, 1))
    LPW2 = np.reshape(tpw_sample['LPW2'], (810000, 1))
    LPW3 = np.reshape(tpw_sample['LPW3'], (810000, 1))
    TPW = np.reshape(tpw_sample['TPW'], (810000, 1))
    sta_index_rr = np.concatenate((sta_index_rr, RR[sta_index]), axis=1)
    sta_index_lpw1 = np.concatenate((sta_index_lpw1, LPW1[sta_index]), axis=1)
    sta_index_lpw2 = np.concatenate((sta_index_lpw2, LPW2[sta_index]), axis=1)
    sta_index_lpw3 = np.concatenate((sta_index_lpw3, LPW3[sta_index]), axis=1)
    sta_index_tpw = np.concatenate((sta_index_tpw, TPW[sta_index]), axis=1)

CPU times: user 8.21 s, sys: 352 ms, total: 8.57 s
Wall time: 8.82 s


#### csv 포맷으로 관측소별 천리안 위성영상자료 저장

In [25]:
for i, value in enumerate(sta_index):
    date_df = pd.DataFrame(
    {'date': pd.date_range(start='2020-04-01 00:00:00', end='2020-04-01 23:50:00', freq='10min'),
    }
    )
    date_df['RR'] = sta_index_rr[i][1:]
    date_df['LPW1'] = sta_index_lpw1[i][1:]
    date_df['LPW2'] = sta_index_lpw2[i][1:]
    date_df['LPW3'] = sta_index_lpw3[i][1:]
    date_df['TPW'] = sta_index_tpw[i][1:]

    date_df.to_csv(str(value)+".csv")